In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!pip install -q pyyaml==5.1
## solución alternativa: instale la versión anterior de pytorch ya que detectron2 no ha lanzado paquetes para pytorch 1.9 (problema: https://github.com/facebookresearch/detectron2/issues/3158)
!pip install -q torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html --no-cache-dir

# instalar detectron2 que coincida con pytorch 1.8
# Consulte https://detectron2.readthedocs.io/tutorials/install.html para obtener instrucciones
#!pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
!python -m pip install -q 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
pip install transformers datasets

In [ ]:
!pip install sentencepiece

In [ ]:
#instalar doctr c/pytorch
!pip install python-doctr[torch]
!pip install --user tensorflow-addons==0.8.3
!pip install --user tensorflow==2.2.0-rc3
!sudo apt-get install fonts-freefont-ttf -y

In [ ]:
!pip install pdf2image
!apt-get install poppler-utils 
!pip install pypdfium2==1.11.0

In [ ]:
!pip install gradio -q

In [7]:
#Aqui se inicia el codigo de procesamiento de texto como tal
%matplotlib inline
import os

# Let's pick the desired backend
# os.environ['USE_TF'] = '1'
os.environ['USE_TORCH'] = '1'

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
from tqdm.auto import tqdm


import tensorflow as tf #Usar con los modelos 
import gradio

#Se importan las funciones de la libreria docTR
from doctr.io import DocumentFile
#from doctr.models import ocr_predictor
from doctr.models import resnet18
from doctr.models import ocr_predictor

from pdf2image import convert_from_path



from PIL import Image

from datasets import (Array2D, Array3D, ClassLabel, Dataset, Features,
                      Sequence, Value)
from PIL import Image
from tqdm import tqdm
import sentencepiece
from transformers import (AdamW, LayoutLMv2FeatureExtractor,
                          LayoutLMv2ForSequenceClassification,
                          LayoutLMv2Processor, LayoutLMv2Tokenizer,LayoutXLMTokenizer, LayoutXLMProcessor)
import json

#Requerimientos DocTR

In [2]:
predictor = ocr_predictor(pretrained=True).cuda()

  0%|          | 0/101971449 [00:00<?, ?it/s]

  0%|          | 0/63286381 [00:00<?, ?it/s]

In [3]:
#Funcion para el procesamiento del formato y extraccion de
#las palabras y coordenadas contenidas en el JSON
def text_capture(json_export):

  #inicializamos las listas para captura de palabras y coordenadas
  words = []
  boxes = []

  #Proceso de iteracion para el acceso a niveles y procesar la extraccion del texto
  #(Ver Texto **Estrucura del JSON**)

  for fields in json_export['pages']:

    for subfields in fields['blocks']:

      for subfields2 in subfields['lines']:

        for subfields3 in subfields2['words']:
          
          words.append(subfields3['value'])
          bboxes=[int(x*1000) for x in list(subfields3['geometry'][0] + subfields3['geometry'][1])]
          boxes.append(bboxes)

  return words, boxes

In [4]:
def pdfToImage(url_init,url_final,file_name):
  docPDFtImage=convert_from_path(url_init)
  jpgurl=url_final+file_name+'.jpg'
  for page in docPDFtImage:
    page.save(jpgurl, 'JPEG')
  return jpgurl

In [5]:
def name_url(name):  
  x = name.split("/")
  fileName=x[-1]
  urlFile=name[:-len(x[-1])]
  return fileName,urlFile


#Requerimientos LayoutMLX

In [9]:
#Diccionario y lista de unicos labels
f = open('/content/drive/MyDrive/Colab Notebooks/LayoutMXL/models/complementos/id2l.json')
id2label=json.load(f)
labels_unique=list(id2label.values())    
id2label

{'0': 'CienciasSociales',
 '1': 'MatematicasEstadistica',
 '2': 'CienciasSalud',
 '3': 'Biologia',
 '4': 'Quimica'}

In [10]:
type(id2label)

dict

In [11]:
#processor
#processor = LayoutXLMTokenizer.from_pretrained("microsoft/layoutxlm-base", revision="no_ocr")
feature_extractor = LayoutLMv2FeatureExtractor(apply_ocr = False)
tokenizer =  LayoutXLMTokenizer.from_pretrained("microsoft/layoutxlm-base")
processor = LayoutXLMProcessor(feature_extractor, tokenizer)

# we need to define custom features
features = Features({
    'image': Array3D(dtype="int64", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': ClassLabel(num_classes=len(labels_unique), names=labels_unique),
})

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/947 [00:00<?, ?B/s]

In [12]:
#modelo
model =  LayoutLMv2ForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/LayoutMXL/models')
#id2label = model.config.id2label

In [27]:
def infer(image, id2label, model, processor,words,boxes):
  # prepare image for the model
  encoded_inputs = processor(image, words, boxes=boxes,return_tensors="pt",  max_length=512)
  #encoded_inputs = processor(image, words, boxes=boxes,return_tensors="pt", return_offsets_mapping=True)
  #offset_mapping = encoded_inputs.pop('offset_mapping')

  # make sure all keys of encoded_inputs are on the same device as the model
  for k,v in encoded_inputs.items():
    encoded_inputs[k] = v.to(model.device)

  # forward pass
  outputs = model(**encoded_inputs)
  logits = outputs.logits
  predicted_class_idx = logits.argmax(-1).item()
  return id2label[str(predicted_class_idx)]

#Requerimientos Gradio

#Ajuste Gradio

In [29]:
def greet(Document):
  try:
    pdf=DocumentFile.from_pdf(Document.name)
    OCR_pdf=predictor(pdf)
    W,B=text_capture(OCR_pdf.export())
    fileName,urlFile=name_url(Document.name)
    url_image=pdfToImage(Document.name,urlFile,fileName[:-4])
    image = Image.open(url_image)
    image = image.convert("RGB")
    Inferencia=infer(image, id2label, model, processor,W,B)
    result=Inferencia
  except:
    result=str("Error al cargar el pdf")
  return result

In [30]:
title = "Demo: LayoutLMLX with spanish dataset and DocTR for OCR"
description = "Demonstration of LayoutLMXL based on Microsoft's Layout, a transformer for next-generation document imaging comprehension tasks. This particular model is adjusted with pdf documents in Spanish in three categories (Ref_Bancaria, Facturas and Articulos). To use it, simply upload a pdf document with only one image. The results will appear in a few seconds. If you want to enlarge the output, right click on it and select 'Open image in new tab'’."
article = "<p style='text-align: center'><a href='https://arxiv.org/abs/2012.14740'>LayoutLMv2: Multi-modal Pre-training for Visually-Rich Document Understanding</a> | <a href='https://github.com/microsoft/unilm'>Github Repo</a> | <a href='https://katanaml.io' target='_blank'>Katana ML</a> | <a href='https://github.com/katanaml/sparrow'>Sparrow Github Repo</a></p><center><img src='https://visitor-badge.glitch.me/badge?page_id=abaranovskij_cord' alt='visitor badge'></center>"
#examples =[['document.png']]

css = ".output_image, .input_image {height: 40rem !important; width: 100% !important;}"

#Demo launch

In [31]:
import gradio as gr


demo = gr.Interface(fn=greet, inputs="file", outputs="text", 
                     title=title,
                     description=description,
                     article=article,
                     css=css,
                     enable_queue=True)

if __name__ == "__main__":
    demo.launch()

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `enable_queue` is deprecated in `Interface()`, please use it within `launch()` instead.
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://36767.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)
